 # RSI

In [2]:
import pandas as pd

def calculate_RSI(data, window=14):
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

def generate_signals(data):
    signals = []
    for i in range(len(data)):
        if pd.notna(data['RSI'][i]):  # Check if RSI value is not NaN
            if data['RSI'][i] < 40:  # RSI is oversold, consider buying
                signals.append(1)  # Signal to buy
            elif data['RSI'][i] > 60:  # RSI is overbought, consider selling
                signals.append(-1)  # Signal to sell
            else:
                signals.append(0)  # No signal
        else:
            signals.append(0)  # No signal if RSI is NaN
    return signals

# Load the data
df = pd.read_csv('ohlc_1h_data.csv')

# Calculate RSI and add it to the DataFrame
df['RSI'] = calculate_RSI(df)

# Generate RSI signals
df['rsi_signal'] = generate_signals(df)

# Implement crossover logic (e.g., if RSI crosses above 60, sell; if RSI crosses below 40, buy)
df['rsi_signal'] = 0
df.loc[df['RSI'] > 70, 'rsi_signal'] = -1
df.loc[df['RSI'] < 30, 'rsi_signal'] = 1

# Print the tail of the DataFrame with RSI and signals
print(df[['RSI', 'rsi_signal']].dropna().tail())

# Save the full DataFrame with RSI and signals
df.dropna().to_csv('signals.csv', index=False)


          RSI  rsi_signal
19  72.873069          -1
20  73.431724          -1
21  70.877676          -1
22  77.205674          -1
23  79.961344          -1


In [19]:
import pandas as pd

def process_data(input_csv, output_csv, start_date, end_date):
    """
    Process 1-minute OHLC data: filter by date range, resample to 1-hour intervals, and save the result.
    
    Parameters:
    - input_csv: str, path to the input CSV file containing 1-minute OHLC data.
    - output_csv: str, path to the output CSV file for saving the resampled data.
    - start_date: str, the start date for filtering the data (format: 'YYYY-MM-DD').
    - end_date: str, the end date for filtering the data (format: 'YYYY-MM-DD').
    """
    # Load the data
    df = pd.read_csv(input_csv)

    # Convert the 'timestamp' column to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Set the 'timestamp' column as the index
    df.set_index('timestamp', inplace=True)

    # Filter the data for the desired date range
    df_date_range = df.loc[start_date:end_date]

    # Resample to 1-hour intervals and aggregate the OHLC data
    ohlc_1h = df_date_range.resample('4H').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last'
    }).dropna()

    # Save the resampled data to a new CSV file
    ohlc_1h.to_csv(output_csv)

    print(f"Resampled data for the date range {start_date} to {end_date} has been saved to '{output_csv}'.")

if __name__ == "__main__":
    # Parameters
    input_csv = 'ohlc_1m_data_last_day.csv'
    output_csv = 'ohlc_4h_data.csv'
    start_date = '2023-01-01'
    end_date = '2024-01-06'
    
    # Execute the data processing function with the provided parameters
    process_data(input_csv, output_csv, start_date, end_date)


Resampled data for the date range 2023-01-01 to 2024-01-06 has been saved to 'ohlc_4h_data.csv'.


C:\Users\MTariq\AppData\Local\Temp\ipykernel_7160\1876405260.py:26: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ohlc_1h = df_date_range.resample('4H').agg({


In [20]:
import pandas as pd
import ta

def calculate_rsi(df, window):
    """
    Calculate RSI using the provided window.
    """
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], window=window).rsi()
    return df

def generate_signals(df, rsi_lower, rsi_upper):
    """
    Generate Buy/Sell signals based on RSI thresholds and filter out 'Hold' signals.
    """
    df['rsi_signal'] = df['RSI'].apply(lambda x: 1 if x < rsi_lower else (-1 if x > rsi_upper else 0))
    df['signals'] = df['rsi_signal'].apply(lambda x: 'Buy' if x == 1 else ('Sell' if x == -1 else ''))
    # Filter out rows where the signal is 'Hold'
    df_filtered = df[df['signals'] != '']
    return df_filtered

def main(input_csv, output_csv, rsi_window=14, rsi_lower=30, rsi_upper=70):
    """
    Main function to process OHLC data, calculate RSI, generate signals, and save the result.
    """
    # Step 1: Load your OHLC data
    df = pd.read_csv(input_csv)
    
    # Step 2: Calculate RSI
    df = calculate_rsi(df, rsi_window)
    
    # Step 3: Generate Buy/Sell signals and filter out 'Hold'
    df_filtered = generate_signals(df, rsi_lower, rsi_upper)
    
    # Save the result to a new CSV file
    df_filtered.to_csv(output_csv, index=False)
    
    print(f"RSI signals and additional columns have been applied and saved to '{output_csv}'.")

if __name__ == "__main__":
    # Parameters
    input_csv = 'ohlc_4h_data.csv'
    output_csv = '4h_rsi_signals.csv'
    rsi_window = 14
    rsi_lower = 30
    rsi_upper = 70
    
    # Execute the main function with the provided parameters
    main(input_csv, output_csv, rsi_window, rsi_lower, rsi_upper)


RSI signals and additional columns have been applied and saved to '4h_rsi_signals.csv'.
